In [10]:
# -----------------------------
# Main-Agent Ready Insight Agent
# -----------------------------

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, Markdown
from datetime import datetime

class InsightAgent:
    def __init__(self, base_path):
        self.BASE = base_path
        self._load_data()
        self._preprocess_data()
        self._cache_metrics()
        # For live trip updates
        self.live_updates = pd.DataFrame(columns=['trip_id','fuel_used_l','co2_kg','timestamp'])
        # Notifications storage
        self.notifications = pd.DataFrame(columns=['trip_id','driver_id','vehicle_id','channel','message','status','created_at'])

    # -------------------
    # Load CSV files
    # -------------------
    def _load_data(self):
        self.trips = pd.read_csv(f"{self.BASE}\\trips_india.csv", parse_dates=['start_time','end_time','created_at'])
        self.assignments = pd.read_csv(f"{self.BASE}\\assignments_india.csv", parse_dates=['scheduled_start','scheduled_end','created_at','updated_at'])
        self.vehicles = pd.read_csv(f"{self.BASE}\\vehicle_agent_india_full.csv")
        self.drivers = pd.read_csv(f"{self.BASE}\\driver_agent_india_full.csv")
        self.environment = pd.read_csv(f"{self.BASE}\\environment_agent_india_full.csv", parse_dates=['time_window_start','time_window_end','last_updated'])
        self.co2_aggregates = pd.read_csv(f"{self.BASE}\\co2_aggregates_india.csv")
        self.fuel_predictions = pd.read_csv(f"{self.BASE}\\fuel_predictions_india.csv", parse_dates=['created_at'])

    # -------------------
    # Preprocessing
    # -------------------
    def _preprocess_data(self):
        self.trips['start_time'] = pd.to_datetime(self.trips['start_time'])
        self.trips['end_time'] = pd.to_datetime(self.trips['end_time'])

        self.df = self.trips.merge(self.assignments, on='assignment_id', how='left') \
                            .merge(self.drivers, on='driver_id', how='left') \
                            .merge(self.vehicles, on='vehicle_id', how='left')

        # Core metrics
        self.df['fuel_efficiency'] = self.df.apply(
            lambda r: r['fuel_used_l']/r['distance_km'] if r['distance_km']>0 else np.nan, axis=1
        )
        self.df['co2_per_km'] = self.df.apply(
            lambda r: r['co2_kg']/r['distance_km'] if r.get('co2_kg', np.nan) and r['distance_km']>0 else np.nan, axis=1
        )
        self.df['avg_speed_kmph'] = self.df.apply(
            lambda r: (r['distance_km']/(r['duration_min']/60)) if r['duration_min']>0 else np.nan, axis=1
        )
        self.df['trip_efficiency'] = self.df['distance_km'] / self.df['fuel_used_l'].replace(0,np.nan)

        # Extended KPIs
        for col in ['idle_time_pct','load_factor','eco_score']:
            if col not in self.df.columns:
                self.df[col] = np.nan

    # -------------------
    # Cache driver & vehicle metrics for performance
    # -------------------
    def _cache_metrics(self):
        self._driver_metrics = self.df.groupby('driver_id').agg(
            total_trips=('trip_id', 'count'),
            avg_fuel_efficiency=('fuel_efficiency', 'mean'),
            avg_co2_per_km=('co2_per_km', 'mean'),
            avg_speed_kmph=('avg_speed_kmph', 'mean'),
            avg_trip_efficiency=('trip_efficiency', 'mean'),
            avg_idle_time=('idle_time_pct', 'mean'),
            avg_load_factor=('load_factor', 'mean'),
            avg_eco_score=('eco_score','mean')
        ).reset_index()

        self._vehicle_metrics = self.df.groupby('vehicle_id').agg(
            total_trips=('trip_id', 'count'),
            total_distance=('distance_km', 'sum'),
            avg_fuel_efficiency=('fuel_efficiency', 'mean'),
            avg_co2_per_km=('co2_per_km', 'mean'),
            avg_speed_kmph=('avg_speed_kmph', 'mean'),
            avg_trip_efficiency=('trip_efficiency', 'mean'),
            avg_idle_time=('idle_time_pct', 'mean'),
            avg_load_factor=('load_factor','mean'),
            avg_eco_score=('eco_score','mean')
        ).reset_index()

    # -------------------
    # Incremental update per trip (for Main Agent live integration)
    # -------------------
    def process_live_update(self, trip_id, fuel_used_l=None, co2_kg=None, distance_km=None, duration_min=None, timestamp=None):
        timestamp = timestamp or datetime.now()
        update_dict = {'trip_id': trip_id, 'fuel_used_l': fuel_used_l, 'co2_kg': co2_kg, 'timestamp': timestamp}
        self.live_updates = pd.concat([self.live_updates, pd.DataFrame([update_dict])], ignore_index=True)

        # Update df for this trip
        self._update_trip_metrics(trip_id, fuel_used_l, co2_kg, distance_km, duration_min)

    def _update_trip_metrics(self, trip_id, fuel_used_l=None, co2_kg=None, distance_km=None, duration_min=None):
        idx = self.df[self.df['trip_id'] == trip_id].index
        if len(idx) == 0:
            return  # trip not found

        if fuel_used_l is not None:
            self.df.loc[idx, 'fuel_used_l'] = fuel_used_l
        if co2_kg is not None:
            self.df.loc[idx, 'co2_kg'] = co2_kg
        if distance_km is not None:
            self.df.loc[idx, 'distance_km'] = distance_km
        if duration_min is not None:
            self.df.loc[idx, 'duration_min'] = duration_min

        # Recalculate metrics
        self.df.loc[idx, 'fuel_efficiency'] = self.df.loc[idx].apply(
            lambda r: r['fuel_used_l']/r['distance_km'] if r['distance_km']>0 else np.nan, axis=1
        )
        self.df.loc[idx, 'co2_per_km'] = self.df.loc[idx].apply(
            lambda r: r['co2_kg']/r['distance_km'] if r.get('co2_kg', np.nan) and r['distance_km']>0 else np.nan, axis=1
        )
        self.df.loc[idx, 'avg_speed_kmph'] = self.df.loc[idx].apply(
            lambda r: (r['distance_km']/(r['duration_min']/60)) if r['duration_min']>0 else np.nan, axis=1
        )
        self.df.loc[idx, 'trip_efficiency'] = self.df.loc[idx]['distance_km'] / self.df.loc[idx]['fuel_used_l'].replace(0,np.nan)

        # Update cached metrics
        self._cache_metrics()

    # -------------------
    # Anomalies
    # -------------------
    def detect_anomalies(self, fuel_threshold=0.5, co2_threshold=1.0):
        df = self.df.copy()
        # Vehicle-type dynamic thresholds
        if 'vehicle_type' in df.columns:
            vehicle_avg_fuel = df.groupby('vehicle_type')['fuel_efficiency'].transform('mean')
            vehicle_avg_co2 = df.groupby('vehicle_type')['co2_per_km'].transform('mean')
            df['fuel_thresh'] = np.where(df['vehicle_type'].notna(), vehicle_avg_fuel * 0.8, fuel_threshold)
            df['co2_thresh'] = np.where(df['vehicle_type'].notna(), vehicle_avg_co2 * 1.2, co2_threshold)
        else:
            df['fuel_thresh'] = fuel_threshold
            df['co2_thresh'] = co2_threshold

        anomalies = df[(df['fuel_efficiency'] < df['fuel_thresh']) | (df['co2_per_km'] > df['co2_thresh'])]

        # Generate notifications
        for _, row in anomalies.iterrows():
            notification = {
                'trip_id': row['trip_id'],
                'driver_id': row['driver_id'],
                'vehicle_id': row['vehicle_id'],
                'channel': 'push',
                'message': f"Anomaly detected: Fuel {row['fuel_efficiency']:.2f}, CO2 {row['co2_per_km']:.2f}",
                'status': 'queued',
                'created_at': datetime.now()
            }
            self.notifications = pd.concat([self.notifications, pd.DataFrame([notification])], ignore_index=True)

        # Return JSON-ready dicts for Main Agent
        return anomalies.to_dict(orient='records')

    # -------------------
    # Recommendations
    # -------------------
    def generate_recommendations(self):
        recs = []
        anomalies = self.detect_anomalies()
        if anomalies:
            recs.append(f"{len(anomalies)} trips have abnormal fuel or CO2 usage. Investigate drivers or vehicles.")
        low_eff_drivers = self._driver_metrics[self._driver_metrics['avg_fuel_efficiency'] < 1.0]
        if not low_eff_drivers.empty:
            recs.append(f"{len(low_eff_drivers)} drivers need eco-driving training.")
        low_eff_vehicles = self._vehicle_metrics[self._vehicle_metrics['avg_fuel_efficiency'] < 1.0]
        if not low_eff_vehicles.empty:
            recs.append(f"{len(low_eff_vehicles)} vehicles may require maintenance check.")
        return recs

    # -------------------
    # Fleet & Top performers
    # -------------------
    def calculate_fleet_metrics(self):
        return {
            'Average Fuel Efficiency (km/l)': self.df['fuel_efficiency'].mean(),
            'Average CO2 per km (kg/km)': self.df['co2_per_km'].mean(),
            'Total Distance (km)': self.df['distance_km'].sum(),
            'Average Trip Duration (min)': self.df['duration_min'].mean(),
            'Average Speed (km/h)': self.df['avg_speed_kmph'].mean()
        }

    def driver_performance(self):
        return self._driver_metrics.copy()

    def vehicle_performance(self):
        return self._vehicle_metrics.copy()

    # -------------------
    # Plotting
    # -------------------
    def plot_top_drivers(self, top_n=5):
        df = self._driver_metrics.sort_values('avg_fuel_efficiency', ascending=False).head(top_n)
        plt.figure(figsize=(10,5))
        plt.bar(df['driver_id'], df['avg_fuel_efficiency'], color='green')
        plt.title("Top Drivers by Fuel Efficiency")
        plt.xlabel("Driver ID")
        plt.ylabel("Avg Fuel Efficiency (km/l)")
        plt.show()

    def plot_top_vehicles(self, top_n=5):
        df = self._vehicle_metrics.sort_values('avg_fuel_efficiency', ascending=False).head(top_n)
        plt.figure(figsize=(10,5))
        plt.bar(df['vehicle_id'], df['avg_fuel_efficiency'], color='blue')
        plt.title("Top Vehicles by Fuel Efficiency")
        plt.xlabel("Vehicle ID")
        plt.ylabel("Avg Fuel Efficiency (km/l)")
        plt.show()

    def plot_fleet_trends(self, metric='fuel_efficiency', freq='W'):
        df = self.df.copy()
        df['week'] = df['start_time'].dt.to_period(freq)
        trend = df.groupby('week')[metric].mean()
        plt.figure(figsize=(10,5))
        trend.plot(marker='o')
        plt.title(f"Fleet {metric} Trend ({freq})")
        plt.xlabel("Week")
        plt.ylabel(metric)
        plt.show()

    # -------------------
    # Display dashboard
    # -------------------
    def display_dashboard(self):
        display(Markdown("## 📊 Fleet Summary Metrics"))
        for k,v in self.calculate_fleet_metrics().items():
            display(Markdown(f"- **{k}:** {v:.2f}"))

        display(Markdown("## 🚚 Top Drivers"))
        display(self.driver_performance().head(10))

        display(Markdown("## 🚛 Top Vehicles"))
        display(self.vehicle_performance().head(10))

        display(Markdown("## ⚠️ Anomalies Detected"))
        display(pd.DataFrame(self.detect_anomalies()).head(10))

        display(Markdown("## 💡 Recommendations"))
        for r in self.generate_recommendations():
            display(Markdown(f"- {r}"))

        self.plot_top_drivers()
        self.plot_top_vehicles()
        self.plot_fleet_trends()

    # -------------------
    # Export insights
    # -------------------
    def export_insights(self, driver_file='driver_insights.csv', vehicle_file='vehicle_insights.csv',
                        anomalies_file='anomalies.csv', notifications_file='notifications.csv'):
        self.driver_performance().to_csv(f"{self.BASE}\\{driver_file}", index=False)
        self.vehicle_performance().to_csv(f"{self.BASE}\\{vehicle_file}", index=False)
        pd.DataFrame(self.detect_anomalies()).to_csv(f"{self.BASE}\\{anomalies_file}", index=False)
        self.notifications.to_csv(f"{self.BASE}\\{notifications_file}", index=False)
        print("All CSV files exported successfully.")
